In [2]:
!pip install pyttsx3

In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

# Define your dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, truncation=True, padding=True, max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Example data
train_texts = [
    "I want to transfer money",
    "Can you help me send money?",
    "I need to send funds to my friend",
    "How can I get my account details?",
    "Where can I find information about my account?",
    "Can you tell me my account balance?",
    "I wish to retrieve my account details",
    "How much money is left in my account?",
    "Amount of money left in my account"
]
train_labels = [0, 0, 0, 1, 1, 2, 1, 2,2]  # 0 for transfer money, 1 for get account details, 2 for get account balance

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)  # 3 labels: transfer money, get account details, get account balance

# Create dataset and dataloader
train_dataset = IntentDataset(train_texts, train_labels, tokenizer, max_length=32)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Fine-tune the model
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for epoch in range(3):  # Train for 3 epochs as an example
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# After fine-tuning, use the model for inference
def predict_intent(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=32, return_tensors='pt')
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    if predicted_class == 0:
        return "User wants to transfer money."
    elif predicted_class == 1:
        return "User wants to get account details."
    elif predicted_class == 2:
        return "User wants to get account balance."
    else:
        return "Unable to determine user's intent."


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

c:\Users\DELL\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.we

In [15]:
# Save the fine-tuned model
save_path = "model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('model\\tokenizer_config.json',
 'model\\special_tokens_map.json',
 'model\\vocab.json',
 'model\\merges.txt',
 'model\\added_tokens.json')

In [ ]:
# Load the saved model and tokenizer
save_path = "model"
loaded_model = GPT2ForSequenceClassification.from_pretrained(save_path, num_labels=3)
loaded_tokenizer = GPT2Tokenizer.from_pretrained(save_path)

# Example inference using the loaded model and tokenizer

def predict_intent(text):
    inputs = loaded_tokenizer(text, truncation=True, padding=True, max_length=32, return_tensors='pt')
    outputs = loaded_model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    if predicted_class == 0:
        return "User wants to transfer money."
    elif predicted_class == 1:
        return "User wants to get account details."
    elif predicted_class == 2:
        return "User wants to get account balance."
    else:
        return "Unable to determine user's intent."


In [6]:
!pip install SpeechRecognition

     --------------------------------------- 32.8/32.8 MB 73.7 kB/s eta 0:00:00


In [8]:
!pip install pyaudio

     ------------------------------------ 164.1/164.1 kB 281.2 kB/s eta 0:00:00


In [18]:

predict_intent("hello i want to know my account balance")

'User wants to get account balance.'

In [10]:
import json
import pyttsx3
import speech_recognition as sr

def text_to_speech(text):
    # Initialize the text-to-speech engine
    engine = pyttsx3.init()
    
    # Set properties (optional)
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1)   # Volume (0.0 to 1.0)
    
    # Convert the text to speech and speak it
    engine.say(text)
    
    # Wait for speech to finish
    engine.runAndWait()

def speech_to_text():
    # Initialize recognizer
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        # Convert speech to text
        user_input = recognizer.recognize_google(audio)
        print("User said:", user_input)
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand what you said.")
    except sr.RequestError as e:
        print("Sorry, I couldn't request results. Please check your internet connection.")

def insert_user(name, account_number, balance):
    # Load existing user data from JSON file
    with open('users.json', 'r') as file:
        users = json.load(file)

    # Append new user data
    users.append({
        "name": name,
        "account_number": account_number,
        "balance": balance
    })

    # Write updated user data back to JSON file
    with open('users.json', 'w') as file:
        json.dump(users, file)

def get_user(account_number):
    # Load existing user data from JSON file
    with open('users.json', 'r') as file:
        users = json.load(file)

    # Find user by account number
    for user in users:
        if user["account_number"] == account_number:
            return user

    # Return None if user not found
    return None

def banking_chatbot():
    # Introduction
    text_to_speech("Hello, Welcome to Banking Chatbot. Are you a new user or an existing user?")
    
    # Get user response
    user_response = speech_to_text()
    
    if user_response:
        if "new" in user_response:
            # Ask for new user details
            text_to_speech("Please provide your name.")
            name = input("Name: ")
            text_to_speech("Please provide your account number.")
            account_number = input("Account Number: ")
            text_to_speech("Please provide your account balance.")
            balance = int(input("Account Balance: "))
            
            # Insert new user details into the JSON file
            insert_user(name, account_number, balance)
            
            # Confirmation message
            text_to_speech("Your details have been successfully stored.")
        elif "existing" in user_response:
            # Ask for existing user details
            text_to_speech("Please enter your account number for authentication.")
            account_number = input("Account Number: ")

            # Get user data
            user = get_user(account_number)
            
            if user:
                # Ask for authentication code
                text_to_speech("Please enter your authentication code.")
                authentication_code = input("Authentication Code: ")
                
                # Dummy authentication check (replace with actual authentication mechanism)
                if authentication_code == "1234":
                    text_to_speech("Authentication successful.")
                    
                    # Present service options to the user
                    text_to_speech("I provide three types of services: transfer money, get account details, and get account balance. What type of service do you want?")
                    
                    # Get user's desired service
                    service_type = predict_intent(speech_to_text())
                    
                    # Process the service type
                    if "transfer money" in service_type:
                        text_to_speech("You have chosen to transfer money.")
                        # Ask for recipient account number
                        text_to_speech("Please type the account number to which you want to transfer money.")
                        recipient_account_number = input("Recipient Account Number: ")
                        # Get recipient user data
                        recipient_user = get_user(recipient_account_number)
                        if recipient_user:
                            # Ask for transfer amount
                            text_to_speech("Please type the amount you wish to transfer.")
                            transfer_amount = float(input("Transfer Amount: "))
                            # Check if the transfer amount is greater than account balance
                            if transfer_amount <= user["balance"]:
                                # Perform the transaction
                                user["balance"] -= transfer_amount
                                recipient_user["balance"] += transfer_amount
                                # Update user data in JSON file
                                with open('users.json', 'w') as file:
                                    json.dump(user, file)
                                    json.dump(recipient_user, file)
                                text_to_speech("Transaction successful.")
                            else:
                                text_to_speech("Transaction failed. Insufficient balance.")
                        else:
                            text_to_speech("Transaction failed. Recipient account does not exist.")
                    elif "get account details" in service_type:
                        text_to_speech("You have chosen to get account details.")
                        # Retrieve and display account details
                        text_to_speech(f"Name: {user['name']}")
                        text_to_speech(f"Account Number: {user['account_number']}")
                        text_to_speech(f"Account Balance: {user['balance']}")
                    elif "get account balance" in service_type:
                        text_to_speech("You have chosen to get account balance.")
                        # Display account balance
                        text_to_speech(f"Your account balance is {user['balance']}.")
                    else:
                        text_to_speech("Sorry, I couldn't understand your service choice.")
                else:
                    text_to_speech("Sorry, authentication failed. Please try again.")
            else:
                text_to_speech("Sorry, the account number provided does not exist.")
        else:
            text_to_speech("Sorry, I couldn't understand your response. Please try again.")
    else:
        text_to_speech("Sorry, I couldn't understand you. Please try again.")

# Call the function to start the banking chatbot
banking_chatbot()


Listening...
Recognizing...
User said: existing user
Listening...
Recognizing...
User said: I want to know how much money is left in my account
